In [1]:
import requests
import json
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('combined_scopus_master_combined_with_match_2205.xlsx')

In [9]:
def reference_formatter(eid):
    resp = requests.get(f"https://api.elsevier.com/content/abstract/eid/{eid}",
                        headers={'Accept':'application/json',
                                 'X-ELS-APIKey': '6545798661bccd5ab7ef446593a92c2f'})

    if ('abstracts-retrieval-response' in resp.json()
       and resp.json()['abstracts-retrieval-response'] is not None 
       and 'item' in resp.json()['abstracts-retrieval-response'] 
       and resp.json()['abstracts-retrieval-response']['item'] is not None
       and resp.json()['abstracts-retrieval-response']['item']['bibrecord'] is not None 
       and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail'] is not None
       and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography'] is not None 
       and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'] is not None):
        
        result_ref = ""
        for i in range(len(resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'])):
            formatted_reference = ""
            ref_author = False
            ref_pubyear = False
            ref_title = False
            
            #ref author - first author
            if (resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info'] is not None
                and 'ref-authors' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']
                and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-authors'] is not None
                and 'author' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-authors']
                and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-authors']['author'] is not None
                and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-authors']['author'][0] is not None
                and 'ce:indexed-name' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-authors']['author'][0]):
                formatted_reference += resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-authors']['author'][0]['ce:indexed-name']
                formatted_reference += ", "
                ref_author = True
                
            #ref publication year
            if (resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info'] is not None
                and 'ref-publicationyear' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']
                and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-publicationyear'] is not None
                and '@first' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-publicationyear']
                and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-publicationyear']['@first'] is not None):
                formatted_reference += resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-publicationyear']['@first']
                formatted_reference += ", "
                ref_pubyear = True
                
            #ref title
            if (resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info'] is not None
                and 'ref-title' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']
                and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-title'] is not None
                and 'ref-titletext' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-title']
                and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-title']['ref-titletext'] is not None):
                formatted_reference += resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-title']['ref-titletext']
                formatted_reference += ", "
                ref_title = True
            
            if (ref_author and ref_pubyear and ref_title):
                #ref sourcetitle
                if (resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info'] is not None
                    and 'ref-sourcetitle' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']
                    and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-sourcetitle'] is not None):
                    formatted_reference += resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-sourcetitle']
                    formatted_reference += ", "

                #ref volume
                if (resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info'] is not None
                    and 'ref-volisspag' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']
                    and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag'] is not None
                    and 'voliss' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']
                    and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['voliss'] is not None
                    and '@volume' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['voliss']
                    and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['voliss']['@volume'] is not None):
                    formatted_reference += resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['voliss']['@volume']
                    formatted_reference += ", "

                #ref pagerange
                if (resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info'] is not None
                    and 'ref-volisspag' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']
                    and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag'] is not None
                    and 'pagerange' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']
                    and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['pagerange'] is not None
                    and '@first' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['pagerange']
                    and '@last' in resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['pagerange']
                    and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['pagerange']['@first'] is not None
                    and resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['pagerange']['@last'] is not None):
                    formatted_reference += resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['pagerange']['@first']
                    formatted_reference += "-"
                    formatted_reference += resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-info']['ref-volisspag']['pagerange']['@last']

            else:
                formatted_reference = ""
                
            formatted_reference.replace(";", "")
            if(len(formatted_reference) >=2 and formatted_reference[-2] == ","):
                formatted_reference = formatted_reference[:-2]
                                          
            if(len(formatted_reference) == 0):
                    formatted_reference += resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][i]['ref-fulltext'].replace(";", "")
            
            formatted_reference.replace(";", "")
            result_ref += formatted_reference
            result_ref += ";"
            #print(formatted_reference, "\n \n")
            
        #print(result_ref)
        result_ref = result_ref[:-1];
        return result_ref    
    return ""

In [10]:
eid=  "2-s2.0-85114075647"
print(reference_formatter(eid))

In [8]:
resp = requests.get(f"https://api.elsevier.com/content/abstract/eid/2-s2.0-85114075647",
                        headers={'Accept':'application/json',
                                 'X-ELS-APIKey': '6545798661bccd5ab7ef446593a92c2f'})
(resp.json()['abstracts-retrieval-response']['item']['bibrecord']['tail']['bibliography']['reference'][2], "\n")


KeyError: 'item'

In [43]:
formatted_references_column = []
count = 0
for i in df['eid']:
    count +=1
    print(count, i)
    formatted_references_column.append(reference_formatter(i))
    #print(formatted_references_column[-1], "\n")

1 2-s2.0-85099799917
2 2-s2.0-85114075647
3 2-s2.0-85089978670
4 2-s2.0-85108303432
5 2-s2.0-85128295641
6 2-s2.0-85096883387
7 2-s2.0-85092228480
8 2-s2.0-85089897949
9 2-s2.0-85083103707
10 2-s2.0-85083091181
11 2-s2.0-85089888240
12 2-s2.0-85091301104
13 2-s2.0-85109080849
14 2-s2.0-85096458998
15 2-s2.0-85118617382
16 2-s2.0-85095696357
17 2-s2.0-85115904896
18 2-s2.0-85099721559
19 2-s2.0-85087658362
20 2-s2.0-85087616700
21 2-s2.0-85100595960
22 2-s2.0-85117529402
23 2-s2.0-85122991653
24 2-s2.0-85087616700
25 2-s2.0-85083983127
26 2-s2.0-85084617534
27 2-s2.0-85084617294
28 2-s2.0-85091502089
29 2-s2.0-85090300911
30 2-s2.0-85083667513
31 2-s2.0-85142495393
32 2-s2.0-85081208379
33 2-s2.0-85083863715
34 2-s2.0-85139865129
35 2-s2.0-85086001510
36 2-s2.0-85098561157
37 2-s2.0-85081660153
38 2-s2.0-85081720938
39 2-s2.0-85101835746
40 2-s2.0-85092420423
41 2-s2.0-85082955449
42 2-s2.0-85103181718
43 2-s2.0-85089679681
44 2-s2.0-85086781288
45 2-s2.0-85125126073
46 2-s2.0-850977220

362 2-s2.0-85086886193
363 2-s2.0-85114521802
364 2-s2.0-85110110235
365 2-s2.0-85094219825
366 2-s2.0-85092512591
367 2-s2.0-85091593723
368 2-s2.0-85090546150
369 2-s2.0-85088812960
370 2-s2.0-85086622903
371 2-s2.0-85119274684
372 2-s2.0-85121477295
373 2-s2.0-85109057108
374 2-s2.0-85109016177
375 2-s2.0-85108386683
376 2-s2.0-85106644551
377 2-s2.0-85106962839
378 2-s2.0-85105784684
379 2-s2.0-85107964025
380 2-s2.0-85107914182
381 2-s2.0-85105285402
382 2-s2.0-85105333202
383 2-s2.0-85105467888
384 2-s2.0-85121477276
385 2-s2.0-85121477104
386 2-s2.0-85121477051
387 2-s2.0-85105554314
388 2-s2.0-85115449327
389 2-s2.0-85116957797
390 2-s2.0-85105094613
391 2-s2.0-85104104418
392 2-s2.0-85104359444
393 2-s2.0-85104352815
394 2-s2.0-85103765918
395 2-s2.0-85104161320
396 2-s2.0-85116957479
397 2-s2.0-85116957390
398 2-s2.0-85111686753
399 2-s2.0-85109316636
400 2-s2.0-85103452247
401 2-s2.0-85102119005
402 2-s2.0-85096081486
403 2-s2.0-85087471317
404 2-s2.0-85122249004
405 2-s2.0-

719 2-s2.0-85101388992
720 2-s2.0-85094631632
721 2-s2.0-85096902367
722 2-s2.0-85118861036
723 2-s2.0-85121291093
724 2-s2.0-85091021220
725 2-s2.0-85089489604
726 2-s2.0-85090216268
727 2-s2.0-85089784099
728 2-s2.0-85100909171
729 2-s2.0-85101034111
730 2-s2.0-85101021074
731 2-s2.0-85101373864
732 2-s2.0-85100983537
733 2-s2.0-85097879909
734 2-s2.0-85097888035
735 2-s2.0-85097768808
736 2-s2.0-85096839907
737 2-s2.0-85096995069
738 2-s2.0-85096222328
739 2-s2.0-85095757499
740 2-s2.0-85095441620
741 2-s2.0-85094961695
742 2-s2.0-85094619318
743 2-s2.0-85100801364
744 2-s2.0-85101288009
745 2-s2.0-85105768464
746 2-s2.0-85100711984
747 2-s2.0-85100539026
748 2-s2.0-85103180030
749 2-s2.0-85099378040
750 2-s2.0-85104110050
751 2-s2.0-85085765745
752 2-s2.0-85085772268
753 2-s2.0-85122960037
754 2-s2.0-85118648353
755 2-s2.0-85112723038
756 2-s2.0-85118619545
757 2-s2.0-85117841191
758 2-s2.0-85116613691
759 2-s2.0-85113218405
760 2-s2.0-85109655396
761 2-s2.0-85108399798
762 2-s2.0-

1073 2-s2.0-85121450689
1074 2-s2.0-85121441817
1075 2-s2.0-85121436667
1076 2-s2.0-85121431588
1077 2-s2.0-85121428256
1078 2-s2.0-85121382324
1079 2-s2.0-85120824284
1080 2-s2.0-85120793089
1081 2-s2.0-85120789548
1082 2-s2.0-85128291531
1083 2-s2.0-85120738755
1084 2-s2.0-85120669013
1085 2-s2.0-85120038117
1086 2-s2.0-85119413036
1087 2-s2.0-85118746900
1088 2-s2.0-85087970185
1089 2-s2.0-85092297266
1090 2-s2.0-85097068914
1091 2-s2.0-85088584187
1092 2-s2.0-85118402688
1093 2-s2.0-85118391104
1094 2-s2.0-85117940333
1095 2-s2.0-85120832127
1096 2-s2.0-85120001544
1097 2-s2.0-85123211995
1098 2-s2.0-85123168990
1099 2-s2.0-85123112662
1100 2-s2.0-85123091411
1101 2-s2.0-85123061246
1102 2-s2.0-85123061225
1103 2-s2.0-85122893792
1104 2-s2.0-85122886727
1105 2-s2.0-85122880627
1106 2-s2.0-85122857729
1107 2-s2.0-85095957206
1108 2-s2.0-85096746333
1109 2-s2.0-85089986198
1110 2-s2.0-85092560539
1111 2-s2.0-85096758540
1112 2-s2.0-85096317599
1113 2-s2.0-85122730101
1114 2-s2.0-8512

1415 2-s2.0-85102527253
1416 2-s2.0-85089656486
1417 2-s2.0-85115229638
1418 2-s2.0-85092504768
1419 2-s2.0-85089298803
1420 2-s2.0-85122935442
1421 2-s2.0-85119298126
1422 2-s2.0-85114418703
1423 2-s2.0-85110524357
1424 2-s2.0-85106884010
1425 2-s2.0-85105746295
1426 2-s2.0-85114295547
1427 2-s2.0-85116129656
1428 2-s2.0-85116124562
1429 2-s2.0-85116046791
1430 2-s2.0-85116045965
1431 2-s2.0-85115792522
1432 2-s2.0-85115761630
1433 2-s2.0-85115431653
1434 2-s2.0-85115246557
1435 2-s2.0-85115204334
1436 2-s2.0-85115116022
1437 2-s2.0-85100299865
1438 2-s2.0-85100074541
1439 2-s2.0-85098552262
1440 2-s2.0-85102325599
1441 2-s2.0-85102768829
1442 2-s2.0-85102640656
1443 2-s2.0-85102517754
1444 2-s2.0-85103267397
1445 2-s2.0-85103237176
1446 2-s2.0-85102735168
1447 2-s2.0-85103561032
1448 2-s2.0-85115003350
1449 2-s2.0-85114747901
1450 2-s2.0-85114492081
1451 2-s2.0-85114459751
1452 2-s2.0-85114236820
1453 2-s2.0-85114227271
1454 2-s2.0-85114222747
1455 2-s2.0-85099309454
1456 2-s2.0-8510

1757 2-s2.0-85113337608
1758 2-s2.0-85116400783
1759 2-s2.0-85087989970
1760 2-s2.0-85088033531
1761 2-s2.0-85088749231
1762 2-s2.0-85086781951
1763 2-s2.0-85105245784
1764 2-s2.0-85106190943
1765 2-s2.0-85116526112
1766 2-s2.0-85097827226
1767 2-s2.0-85097815383
1768 2-s2.0-85097123090
1769 2-s2.0-85097734306
1770 2-s2.0-85098252277
1771 2-s2.0-85099052444
1772 2-s2.0-85097782788
1773 2-s2.0-85086412090
1774 2-s2.0-85086363992
1775 2-s2.0-85085750933
1776 2-s2.0-85085566481
1777 2-s2.0-85085688258
1778 2-s2.0-85083323221
1779 2-s2.0-85100494951
1780 2-s2.0-85100319495
1781 2-s2.0-85119446050
1782 2-s2.0-85119669485
1783 2-s2.0-85118719303
1784 2-s2.0-85101943170
1785 2-s2.0-85118485996
1786 2-s2.0-85117112916
1787 2-s2.0-85100511449
1788 2-s2.0-85099576426
1789 2-s2.0-85112653355
1790 2-s2.0-85130151760
1791 2-s2.0-85112257623
1792 2-s2.0-85111499343
1793 2-s2.0-85116165194
1794 2-s2.0-85115971659
1795 2-s2.0-85114674892
1796 2-s2.0-85097827588
1797 2-s2.0-85097125105
1798 2-s2.0-8510

KeyError: 0

In [44]:
print(len(formatted_references_column))

2028


In [ ]:
formatted_references_column.append("")

In [65]:
(formatted_references_column[2028])
len(formatted_references_column)

2029

In [66]:
count = 2029
error_count = 0
for i in df['eid']:
    error_count +=1
    if(error_count >= 2030):
        count +=1   
        print(count, i)
        formatted_references_column.append(reference_formatter(i))
        #print(formatted_references_column[-1], "\n")   

2030 2-s2.0-85098551430
2031 2-s2.0-85097097358
2032 2-s2.0-85098494787
2033 2-s2.0-85096656153
2034 2-s2.0-85098545130
2035 2-s2.0-85098506199
2036 2-s2.0-85098592458
2037 2-s2.0-85095388248
2038 2-s2.0-85096816735
2039 2-s2.0-85098224971
2040 2-s2.0-85100003375
2041 2-s2.0-85105508611
2042 2-s2.0-85099954925
2043 2-s2.0-85100069053
2044 2-s2.0-85096814678
2045 2-s2.0-85096075775
2046 2-s2.0-85122501404
2047 2-s2.0-85092659164
2048 2-s2.0-85090704847
2049 2-s2.0-85096508402
2050 2-s2.0-85096508312
2051 2-s2.0-85096508488
2052 2-s2.0-85122484779
2053 2-s2.0-85096727030
2054 2-s2.0-85094167493
2055 2-s2.0-85122194504
2056 2-s2.0-85122150748
2057 2-s2.0-85122127557
2058 2-s2.0-85121248667
2059 2-s2.0-85120733853
2060 2-s2.0-85120674717
2061 2-s2.0-85119046251
2062 2-s2.0-85118882178
2063 2-s2.0-85118795611
2064 2-s2.0-85118671620
2065 2-s2.0-85118548474
2066 2-s2.0-85118532551
2067 2-s2.0-85118422744
2068 2-s2.0-85118280230
2069 2-s2.0-85090201947
2070 2-s2.0-85093895839
2071 2-s2.0-8509

2372 2-s2.0-85120943926
2373 2-s2.0-85120893297
2374 2-s2.0-85120769964
2375 2-s2.0-85121618930
2376 2-s2.0-85121578334
2377 2-s2.0-85120179044
2378 2-s2.0-85120156366
2379 2-s2.0-85120740311
2380 2-s2.0-85119127198
2381 2-s2.0-85118755105
2382 2-s2.0-85118747197
2383 2-s2.0-85131636539
2384 2-s2.0-85118722050
2385 2-s2.0-85118533046
2386 2-s2.0-85119581658
2387 2-s2.0-85118890450
2388 2-s2.0-85118841405
2389 2-s2.0-85116539483
2390 2-s2.0-85117319387
2391 2-s2.0-85116556909
2392 2-s2.0-85118261862
2393 2-s2.0-85118255227
2394 2-s2.0-85118233170
2395 2-s2.0-85116797652
2396 2-s2.0-85116762624
2397 2-s2.0-85116575785
2398 2-s2.0-85115640527
2399 2-s2.0-85115152685
2400 2-s2.0-85115081814
2401 2-s2.0-85115071589
2402 2-s2.0-85116565648
2403 2-s2.0-85117794253
2404 2-s2.0-85117215864
2405 2-s2.0-85116969531
2406 2-s2.0-85116558181
2407 2-s2.0-85116540046
2408 2-s2.0-85096081489
2409 2-s2.0-85096029587
2410 2-s2.0-85095800340
2411 2-s2.0-85116700498
2412 2-s2.0-85083673550
2413 2-s2.0-8511

2714 2-s2.0-85128614723
2715 2-s2.0-85125973449
2716 2-s2.0-85127680803
2717 2-s2.0-85125592860
2718 2-s2.0-85126640333
2719 2-s2.0-85126114018
2720 2-s2.0-85126078283
2721 2-s2.0-85127420020
2722 2-s2.0-85129374884
2723 2-s2.0-85125405262
2724 2-s2.0-85126081228
2725 2-s2.0-85127233791
2726 2-s2.0-85130342341
2727 2-s2.0-85127968515
2728 2-s2.0-85126208654
2729 2-s2.0-85124061541
2730 2-s2.0-85128648470
2731 2-s2.0-85128013608
2732 2-s2.0-85134624408
2733 2-s2.0-85123904773
2734 2-s2.0-85129885889
2735 2-s2.0-85128956449
2736 2-s2.0-85129819667
2737 2-s2.0-85128622982
2738 2-s2.0-85125873492
2739 2-s2.0-85128520380
2740 2-s2.0-85125739073
2741 2-s2.0-85128465091
2742 2-s2.0-85128259972
2743 2-s2.0-85123979546
2744 2-s2.0-85127665950
2745 2-s2.0-85125549320
2746 2-s2.0-85123481595
2747 2-s2.0-85123569132
2748 2-s2.0-85123384674
2749 2-s2.0-85127913906
2750 2-s2.0-85125613288
2751 2-s2.0-85124012216
2752 2-s2.0-85127231902
2753 2-s2.0-85125664987
2754 2-s2.0-85125627557
2755 2-s2.0-8512

3056 2-s2.0-85091760777
3057 2-s2.0-85125656463
3058 2-s2.0-85124101635
3059 2-s2.0-85094860971
3060 2-s2.0-85098273986
3061 2-s2.0-85135551165
3062 2-s2.0-85133336939
3063 2-s2.0-85132193534
3064 2-s2.0-85134546279
3065 2-s2.0-85134162832
3066 2-s2.0-85130025958
3067 2-s2.0-85133619387
3068 2-s2.0-85131903543
3069 2-s2.0-85132081975
3070 2-s2.0-85134172133
3071 2-s2.0-85130918056
3072 2-s2.0-85135224125
3073 2-s2.0-85134589144
3074 2-s2.0-85135189250
3075 2-s2.0-85130048627
3076 2-s2.0-85134342487
3077 2-s2.0-85132122713
3078 2-s2.0-85134628407
3079 2-s2.0-85130023835
3080 2-s2.0-85130825279
3081 2-s2.0-85131220159
3082 2-s2.0-85134827575
3083 2-s2.0-85132834610
3084 2-s2.0-85129954235
3085 2-s2.0-85126480066
3086 2-s2.0-85127450178
3087 2-s2.0-85127105488
3088 2-s2.0-85130482933
3089 2-s2.0-85130235470
3090 2-s2.0-85131772157
3091 2-s2.0-85133373040
3092 2-s2.0-85131702378
3093 2-s2.0-85132951947
3094 2-s2.0-85133282630
3095 2-s2.0-85135061098
3096 2-s2.0-85135185785
3097 2-s2.0-8513

3398 2-s2.0-85137204592
3399 2-s2.0-85137196214
3400 2-s2.0-85137182728
3401 2-s2.0-85136442062
3402 2-s2.0-85138732386
3403 2-s2.0-85138651619
3404 2-s2.0-85138610546
3405 2-s2.0-85138606811
3406 2-s2.0-85138598790
3407 2-s2.0-85138492179
3408 2-s2.0-85137542215
3409 2-s2.0-85137404308
3410 2-s2.0-85133233424
3411 2-s2.0-85134700423
3412 2-s2.0-85134504683
3413 2-s2.0-85136359814
3414 2-s2.0-85138669159
3415 2-s2.0-85138651182
3416 2-s2.0-85138634856
3417 2-s2.0-85138621898
3418 2-s2.0-85138440501
3419 2-s2.0-85137387950
3420 2-s2.0-85137387597
3421 2-s2.0-85137381417
3422 2-s2.0-85137337327
3423 2-s2.0-85137138987
3424 2-s2.0-85137138862
3425 2-s2.0-85137138680
3426 2-s2.0-85137138477
3427 2-s2.0-85137138180
3428 2-s2.0-85137138125
3429 2-s2.0-85138440081
3430 2-s2.0-85138438086
3431 2-s2.0-85138437065
3432 2-s2.0-85131049738
3433 2-s2.0-85137135769
3434 2-s2.0-85137135628
3435 2-s2.0-85137055672
3436 2-s2.0-85127944454
3437 2-s2.0-85128339975
3438 2-s2.0-85124424259
3439 2-s2.0-8512

3740 2-s2.0-85128229965


In [71]:
len(formatted_references_column)
formatted_references_column[-1]

'Huang C., 2020, Clinical features of patients infected with 2019 novel coronavirus in Wuhan, China, Lancet, 395, 497-506;Kanji J.N., 2021, False negative rate of COVID-19 PCR testing: a discordant testing analysis, Virol J, 18;Ji T., 2020, Detection of COVID-19: a review of the current literature and future perspectives, Biosens Bioelectron, 166;Tokyo Metropolitan Government. COVID-19 information Website. Updates on COVID-19, https://stopcovid19.metro.tokyo.lg.jp/en/reference, [accessed 22 December 2021]. In: Tokyo: Other Indexes.;US Centers for Disease Control and Prevention. Interim guidelines for clinical specimens for COVID-19. https://www.cdc.gov/coronavirus/2019-ncov/lab/guidelines-clinical-specimens.html. (Accessed 17 December 2021);Abbott, Laboratories, ID-NOW COVID-19 quick reference instructions. https://dam.abbott.com/en-us/homepage/coronavirus/38993-ID-NOW-QRG-r4-HD.pdf. (Accessed 28 December 2021);Xpert®, Cepheid, Xpress SARS-CoV-2 instructions for use. https://www.fda.go

In [72]:
df

,ID,STUDY TITLE/NAME,STUDY DESCRIPTION,STUDY METHODOLOGY,PUBLICATION DATE,SAMPLE SIZE,TEST MANUFACTURER,TEST MANUFACTURER.1,TEST MANUFACTURER.2,LINK TO PUBLICATION,...,author_y,authkeywords_y,article-number_y,source-id_y,openaccessFlag_y,Affliation-name_y,fund-sponsor_y,link ref=self_y,link ref=scopus_y,link ref=scopus-citedby_y
0,1,Rapid Serological Tests Have a Role in Asympto...,Viva-Diag Rapid anti-SARS-CoV-2 IgM-positive v...,Testing Protocols,4/1/2020 0:00,525,Other - Academic PCR Lab,NaN,NaN,https://www.medrxiv.org/content/10.1101/2020.0...,...,"Mullins K.E.,Merrill V.R.,Ward M.,King B.,Rock...",COVID-19 | SARS-CoV-2 | Serology,NaN,1.695400e+04,0.0,"University of Maryland Medical System,Universi...",NaN,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...
1,2,Evaluation of a Rapid Diagnostic Assay for Det...,COVID-19 Ag Respi-Strip was compared with PCR ...,Test Validation,7/1/2020 0:00,138,Other - Commercial PCR Lab,NaN,NaN,https://jcm.asm.org/content/58/8/e00977-20,...,"Seynaeve Y.,Heylen J.,Fontaine C.,Maclot F.,Me...",Antigen detection | COVID-19 | Rapid diagnosti...,2774,2.110105e+10,1.0,Université de Liège,NaN,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...
2,3,Clinical Course and Molecular Viral Shedding f...,PCR was used to determine Cycle threshold (Ct)...,Community/Epidemiological,8/1/2020 0:00,303,NaN,NaN,NaN,https://jamanetwork.com/journals/jamainternalm...,...,"Lee S.,Kim T.,Lee E.,Lee C.,Kim H.,Rhee H.,Par...",NaN,NaN,2.110023e+10,1.0,"McMaster University, Faculty of Health Science...",Pfizer,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...
3,4,"Clinical Impact, Costs, and Cost-Effectiveness...",Projections of clinical and economic impact of...,Testing Protocols,9/1/2020 0:00,NaN,NaN,NaN,NaN,https://academic.oup.com/cid/advance-article/d...,...,"Paltiel A.D.,Zheng A.,Sax P.E.",NaN,NaN,2.657400e+04,0.0,"Brigham and Women's Hospital,Yale University,H...",National Institutes of Health,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...
4,5,DETECT study: Wearable Fitness Devices Can Imp...,Mobile app to collect smartwatch and activity ...,Technology Validation,10/1/2020 0:00,30529,NaN,NaN,NaN,https://www.nature.com/articles/s41591-020-1123-x,...,"Galende N.,Redondo I.,Dosil-Santamaria M.,Ozam...",age | compliance factors | information campaig...,4853,1.449890e+05,1.0,Universidad del Pais Vasco,NaN,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3735,3831,The Panbio COVID-19 Ag Rapid Test: Which Perfo...,The Panbio COVID-19 Ag Rapid test was compared...,Test Comparison,2021-11-01 00:00:00,81,Abbott,Other - Commercial PCR Lab,NaN,https://pubmed.ncbi.nlm.nih.gov/34758228/,...,"Farfour E.,Amiel C.,Jolly E.,Zia-Chahabi S.,Ma...",Asymptomatic | Contact | COVID-19 | Healthcare...,NaN,2.544600e+04,0.0,Hopital Foch,NaN,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...
3736,3832,Duplex Reverse Transcription Multienzyme Isoth...,"Here, we designed duplex one-step RT-MIRA with...",Test Validation,2021-10-01 00:00:00,NaN,Other,NaN,NaN,https://www.frontiersin.org/articles/10.3389/f...,...,"Shang M.,Li J.,Sun X.,Su N.,Li B.,Jiang Y.,Cho...",Isothermal amplification | Multienzyme isother...,NaN,2.544600e+04,0.0,Chongqing Medical University,NaN,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://w

In [73]:
df['References_formatted'] = formatted_references_column

In [74]:
df

,ID,STUDY TITLE/NAME,STUDY DESCRIPTION,STUDY METHODOLOGY,PUBLICATION DATE,SAMPLE SIZE,TEST MANUFACTURER,TEST MANUFACTURER.1,TEST MANUFACTURER.2,LINK TO PUBLICATION,...,authkeywords_y,article-number_y,source-id_y,openaccessFlag_y,Affliation-name_y,fund-sponsor_y,link ref=self_y,link ref=scopus_y,link ref=scopus-citedby_y,References_formatted
0,1,Rapid Serological Tests Have a Role in Asympto...,Viva-Diag Rapid anti-SARS-CoV-2 IgM-positive v...,Testing Protocols,4/1/2020 0:00,525,Other - Academic PCR Lab,NaN,NaN,https://www.medrxiv.org/content/10.1101/2020.0...,...,COVID-19 | SARS-CoV-2 | Serology,NaN,1.695400e+04,0.0,"University of Maryland Medical System,Universi...",NaN,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...,"Lerner A.M., 2020, The COVID-19 Serology Studi..."
1,2,Evaluation of a Rapid Diagnostic Assay for Det...,COVID-19 Ag Respi-Strip was compared with PCR ...,Test Validation,7/1/2020 0:00,138,Other - Commercial PCR Lab,NaN,NaN,https://jcm.asm.org/content/58/8/e00977-20,...,Antigen detection | COVID-19 | Rapid diagnosti...,2774,2.110105e+10,1.0,Université de Liège,NaN,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...,"Zhu N., 2020, A Novel Coronavirus from Patient..."
2,3,Clinical Course and Molecular Viral Shedding f...,PCR was used to determine Cycle threshold (Ct)...,Community/Epidemiological,8/1/2020 0:00,303,NaN,NaN,NaN,https://jamanetwork.com/journals/jamainternalm...,...,NaN,NaN,2.110023e+10,1.0,"McMaster University, Faculty of Health Science...",Pfizer,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...,"Zhu N., 2020, A novel coronavirus from patient..."
3,4,"Clinical Impact, Costs, and Cost-Effectiveness...",Projections of clinical and economic impact of...,Testing Protocols,9/1/2020 0:00,NaN,NaN,NaN,NaN,https://academic.oup.com/cid/advance-article/d...,...,NaN,NaN,2.657400e+04,0.0,"Brigham and Women's Hospital,Yale University,H...",National Institutes of Health,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...,"Atkenson A, Droste M, Stock J. Economic benefi..."
4,5,DETECT study: Wearable Fitness Devices Can Imp...,Mobile app to collect smartwatch and activity ...,Technology Validation,10/1/2020 0:00,30529,NaN,NaN,NaN,https://www.nature.com/articles/s41591-020-1123-x,...,age | compliance factors | information campaig...,4853,1.449890e+05,1.0,Universidad del Pais Vasco,NaN,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...,"Chauhan S., 2020, Comprehensive review of coro..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3735,3831,The Panbio COVID-19 Ag Rapid Test: Which Perfo...,The Panbio COVID-19 Ag Rapid test was compared...,Test Comparison,2021-11-01 00:00:00,81,Abbott,Other - Commercial PCR Lab,NaN,https://pubmed.ncbi.nlm.nih.gov/34758228/,...,Asymptomatic | Contact | COVID-19 | Healthcare...,NaN,2.544600e+04,0.0,Hopital Foch,NaN,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/citedby.uri?part...,"Lechien JR, 2020, Olfactory and gustatory dysf..."
3736,3832,Duplex Reverse Transcription Multienzyme Isoth...,"Here, we designed duplex one-step RT-MIRA with...",Test Validation,2021-10-01 00:00:00,NaN,Other,NaN,NaN,https://www.frontiersin.org/articles/10.3389/f...,...,Isothermal amplification | Multienzyme isother...,NaN,2.544600e+04,0.0,Chongqing Medical University,NaN,https://api.elsevier.com/content/abstract/scop...,https://www.scopus.com/inward/record.uri?partn...,https://www.scopus.com/inward/

In [75]:
df.to_excel('scopus_master_with_references_formatted_3005_noedit.xlsx',index=False)

C:\Users\sjunuthu\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://www.who.int/emergencies/diseases/novel-coronavirus-2019.;Dong%20E.,%202020,%20An%20interactive%20web-based%20dashboard%20to%20track%20COVID-19%20in%20real%20time,%20Lancet%20Infect%20Dis,%2020,%20533-534;Shang%20J.,%202020,%20Structural%20basis%20of%20receptor%20recognition%20by%20SARS-CoV-2,%20Nature,%20581,%20221-224;Shang%20J.,%202020,%20Cell%20entry%20mechanisms%20of%20SARS-CoV-2,%20Proc%20Natl%20Acad%20Sci%20USA,%20117,%2011727-11734;Du%20L.,%202009,%20The%20spike%20protein%20of%20SARS-CoV–a%20target%20for%20vaccine%20and%20therapeutic%20development,%20Nat%20Rev%20Microbiol,%207,%20226-236;Jia%20H.P.,%202005,%20ACE2%20receptor%20expression%20and%20severe%20acute%20respiratory%20syndrome%20coronavirus%20infection%20depend%20on%20differentiation%20of%20human%20airway%20epithelia,%20J%20Virol,%2079,%2014614-14621;Krammer%20F.,%202020,%20SARS-CoV-2%20vaccines%20in%20developme